# SQLite lesson 1

Whenever we write programs, we need a way to store the data that they contain. One of the most popular ways of doing this is by using relational databases. Structured Query Language (SQL) is the language used by relational databases.

SQL is a separate language from Python, but here we will be using them together. All the SQL statements are going to be fed to Python as string objects.

A database is simply a collection of data about a particular topic, like riders on a bus or patients at a doctor's office, or customers at a store.

Python comes with a module called sqlite3, which allows it to communicate with a relational database to store data and reopen it after a program is closed. This lesson will teach you the basics of working with SQL using Python. 

Let's start with a simple example where we will create a table, add data to it, and then delete the data and quit.

In [1]:
#Let's load the sqlite3 module
import sqlite3 #imports the module sqlite3

In [3]:
#to keep things simple, we will use a database in the computer's memory for this exercise
db = sqlite3.connect(':memory:')

What have we just done here? We've imported the module sqlite3. Then, we have created a connection object to create an in-memory database. Since the database doesn't exist, we have created a new one.

In [5]:
#Let's create a cursor that let's us communicate with the database
cur = db.cursor()

In [9]:
#Let's look at the object types. As you can see, db is a connection to the database, while cur is a cursor.
print(type(db))
print(type(cur))

<class 'sqlite3.Connection'>
<class 'sqlite3.Cursor'>


Relational databases like SQL are made up of tables. Now that we've created a database, let's make a table. We do this using our cursor object, by calling the `cur.execute()` method. We feed a sql statement to the cursor. In this case, we will tell it to create a new table by feeding it `create table test (id_name)`.

In [15]:
cur.execute('create table test(id, name)')
#we have just created a new empty table with two fields.

Please note that if you try to create a table with the same name more than once in a database, the sqlite3 module will not allow it. You can delete a table with the `drop table` command. To erase the table and start over, type `cur.execute('drop table test')` and then run `create table test (id, name)` again.

In [17]:
#Now let's insert some data
cur.execute('insert into test (id, name) values (1, "Thomas")')

In [25]:
#Now let's get the data that we added to it. We do this using the 'select' statement
result = cur.execute('select * from test')
#once we have selected the data, we display it using the fetchall() method
print(result.fetchall()) #the fetchall method returns all the results

[(1, 'Thomas'), (2, 'Debra')]


In [23]:
#Let's add some more data
cur.execute('insert into test (id, name) values (2, "Debra")')

In [27]:
cur.execute('select * from test')
print(cur.fetchall())

[(1, 'Thomas'), (2, 'Debra')]


In [32]:
#Now let's unlock the power of relational databases by performing a query that returns only one value
#In this case we can return the value of the name "Debra" from this query.
cur.execute('select * from test where name like ?', ('Debra',))
cur.fetchall()

[(2, 'Debra')]

What if we wanted to do more than that? Like by adding a function to add values to the database, and another function to delete values from it? How could we do that?

In [42]:
#function to add data
def add_entry(cursor: sqlite3.Connection.cursor, id: int, name: str):
    cursor.execute('insert into test (id, name) values(?, ?)', (id, name))

In [43]:
add_entry(cur, id=3, name='Deborah')

The type hints in the function above are purely cosmetic. You can also write the code like this:

`def add_entry(cursor, id, name):
    cursor.execute('insert into test (id, name) values(?, ?)', (id, name))`

In [44]:
#We now have two entries where the names are similar. Let's fetch all the values to show this
cur.execute('select * from test')
cur.fetchall()

[(1, 'Thomas'), (2, 'Debra'), (3, 'Deborah')]

In [46]:
#What happens if we try to select the entries that are similar to "Deb" in the database
cur.execute('select * from test where name like "Deb"')
cur.fetchall() #no values are returned

[]

Clearly, the like keyword doesn't work. As it turns out, sqlite lets us use `%` as a way of searching based on incomplete commands. This helps us to find results based on partial values. We place the % at the beginning or end of a query depending on what values we currently don't know.

In [55]:
cur.execute('select * from test where name like "%deb%"')
cur.fetchall()

[(2, 'Debra'), (3, 'Deborah')]

In [59]:
#Let's create a function to do a search for us
def findData(cursor: sqlite3.Connection.cursor, aString: str):
    cursor.execute('select * from test where name like ?', (aString,))
    return cur.fetchall()

In [61]:
findData(cur, '%d%')

[(2, 'Debra'), (3, 'Deborah')]

In [62]:
findData(cur, '%t%')

[(1, 'Thomas')]

In [63]:
add_entry(cur, 4, 'Laura')

In [66]:
#Here's the difference in the use of % by position
#before the string we use to search
print(findData(cur, '%L')) #nothing found
#after the string we use to search
print(findData(cur, 'L%'))

[]
[(4, 'Laura')]


Now that we have experimented a bit with the database, feel free to create your own table. 

Since we are done for now, let's delete the tables and close the database. Note that these commands will raise an exception if you try to enter them more than once. Check the appendix for how to get around this with a try...except structure.

In [86]:
#These commands will raise an exception if entered more than once
cur.execute('delete from test')
cur.execute('drop table test')
cur.close()
db.commit() #commit any changes
db.close() #close the connection

ProgrammingError: Cannot operate on a closed cursor.

# Avoid exceptions when closing a database

To get around this, we can use a try...except structure. We can import the exceptions from the sqlite3 module.

Keep in mind you do not have to do this. But it can help you prevent your program from crashing.
`OperationalError` is for when an illegal operation is done in a database, like adding to a table that does not exist. `ProgrammingError` is for when we attempt to operate on a closed database.

In [77]:
from sqlite3 import ProgrammingError, OperationalError
try:
    cur.execute('delete from test') #we are emptying the test table
except OperationalError:
    print('table does not exist')
except ProgrammingError:
    print('Cursor object already closed')

Cursor object already closed


In [79]:
#now the test table is empty.
try:
    cur.execute('drop table test')
except OperationalError:
    print('table does not exist')
except ProgrammingError:
    print('cursor already closed')

cursor already closed


Now, we will perform the final two steps to close our connection to the database. First, we will close the cursor object we have been using. Second, we will commit any changes to the database. Finally, we will close the connection to the database itself. Since we have an in-memory database, this will delete the database for us.

In [81]:
try:
    cur.close()
except ProgrammingError:
    print('cursor already closed')

cursor already closed


In [82]:
try:
    db.commit()
except ProgrammingError:
    print('Database already closed')

Database already closed


In [84]:
db.close()